In [32]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
import seaborn as sns
from scipy.stats.mstats import winsorize
import matplotlib.pyplot as plt
from scipy.stats.stats import ttest_ind
%matplotlib inline

warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'studentsperformance'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df = pd.read_sql_query('select * from studentsperformance',con=engine)

engine.dispose()

print(df.head())

   gender race/ethnicity parental level of education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test preparation course  math score  reading score  writing score  
0                    none          72             72             74  
1               completed          69             90             88  
2                    none          90             95             93  
3                    none          47             57             44  
4                    none          76             78             75  


1. Are there any differences between the genders, ethnicities, and parental level of education with respect to their performances in exams?

In [58]:
metrics = ['math score', 'reading score', 'writing score']

gen_mean = df.groupby('gender')[metrics].mean()
print(gen_mean)

ttest_ind(df[df['gender'] == 'male'][metrics], df[df['gender'] == 'female'][metrics])

        math score  reading score  writing score
gender                                          
female   63.633205      72.608108      72.467181
male     68.728216      65.473029      63.311203


Ttest_indResult(statistic=array([ 5.38324587, -7.95930801, -9.97955791]), pvalue=array([9.12018555e-08, 4.68053874e-15, 2.01987771e-22]))

It looks like females are more successful at reading and writing and males are more successful with math. The results seem to be statistically significan.

In [55]:
eth_mean = df.groupby('race/ethnicity')[metrics].mean()
eth = df['race/ethnicity'].unique()
print(eth_mean)

for var in metrics:
    print('')
    print('---------------------------------------------------------------')
    print('{} T-Test Results'.format(var))
    print('---------------------------------------------------------------')
    for i in range(0, len(eth)):
        for j in range(i+1, len(eth)):
            print('T-Test between {} and {}:'.format(eth[i], eth[j]))
            print(ttest_ind(df[df['race/ethnicity'] == eth[i]][var],
            df[df['race/ethnicity'] == eth[j]][var]))
            

                math score  reading score  writing score
race/ethnicity                                          
group A          61.629213      64.674157      62.674157
group B          63.452632      67.352632      65.600000
group C          64.463950      69.103448      67.827586
group D          67.362595      70.030534      70.145038
group E          73.821429      73.028571      71.407143

---------------------------------------------------------------
math score T-Test Results
---------------------------------------------------------------
T-Test between group B and group C:
Ttest_indResult(statistic=-0.7315669893534263, pvalue=0.4647708939167453)
T-Test between group B and group A:
Ttest_indResult(statistic=0.9355076279747488, pvalue=0.3503415961659957)
T-Test between group B and group D:
Ttest_indResult(statistic=-2.82845539712675, pvalue=0.004886197137104194)
T-Test between group B and group E:
Ttest_indResult(statistic=-6.007397050552227, pvalue=5.007946047497971e-09)
T-Tes

It looks like there are significan difference between the groups except for between groups A and C and groups A and B

In [78]:
par_mean = df.groupby('parental level of education')[metrics].mean()
par = df['parental level of education'].unique()
print(par_mean)

for var in metrics:
    print('')
    print('---------------------------------------------------------------')
    print('{} T-Test Results'.format(var))
    print('---------------------------------------------------------------')
    for i in range(0, len(par)):
        for j in range(i+1, len(par)):
            print('T-Test between {} and {}:'.format(par[i], par[j]))
            print(ttest_ind(df[df['parental level of education'] == par[i]][var],
            df[df['parental level of education'] == par[j]][var]))

                             math score  reading score  writing score
parental level of education                                          
associate's degree            67.882883      70.927928      69.896396
bachelor's degree             69.389831      73.000000      73.381356
high school                   62.137755      64.704082      62.448980
master's degree               69.745763      75.372881      75.677966
some college                  67.128319      69.460177      68.840708
some high school              63.497207      66.938547      64.888268

---------------------------------------------------------------
math score T-Test Results
---------------------------------------------------------------
T-Test between bachelor's degree and some college:
Ttest_indResult(statistic=1.3702352829617477, pvalue=0.17151224218471559)
T-Test between bachelor's degree and master's degree:
Ttest_indResult(statistic=-0.14868196568849312, pvalue=0.8819757824867702)
T-Test between bachelor's degre

Significant differences definitely exist here

2. Are there any differences between the lunch types with respect to their performances in exams? If there are, how do you explain this?

In [64]:
lunch = df.groupby('lunch')[metrics].mean()

ttest_ind(df[df['lunch'] == 'free/reduced'][metrics],
         df[df['lunch'] == 'standard'][metrics])

Ttest_indResult(statistic=array([-11.83718047,  -7.45105647,  -8.0097842 ]), pvalue=array([2.41319560e-30, 2.00279665e-13, 3.18618958e-15]))

It looks like there is a significant difference in success based on type of lunch. This is probably related to the education level of the parents (which also produces significant differences), as more educated parents are more likely to have more money to spend on their kids' lunch.

3. Does the test preparation course seem to have an effect on the exam performances?

In [75]:
prep = df.groupby('test preparation course')[metrics].mean()
print(prep)

ttest_ind(df[df['test preparation course'] == 'completed'][metrics],
         df[df['test preparation course'] == 'none'][metrics])

                         math score  reading score  writing score
test preparation course                                          
completed                 69.695531      73.893855      74.418994
none                      64.077882      66.534268      64.504673


Ttest_indResult(statistic=array([ 5.70461642,  7.87166354, 10.40917344]), pvalue=array([1.53591346e-08, 9.08178334e-15, 3.68529174e-24]))

The test preparation courses do indeed make a difference.

4. Which 2 exam scores are most correlated with each other?

In [77]:
df.corr()

,math score,reading score,writing score
math score,1.000000,0.817580,0.802642
reading score,0.817580,1.000000,0.954598
writing score,0.802642,0.954598,1.000000


Reading and writing are most correlated with each other.